# Feature Extraction

In [1]:
#!wget -c https://github.com/roman-rybalko/telegram-data-clustering-contest/archive/8da391a6b6d952bbf0555e8a80ff864cf9c79bda.zip -O d2.zip
#!unzip -uq d2.zip
#!mkdir dataset
#!mv -v */dataset1/* */dataset2/* ./dataset/

--2019-11-22 20:27:49--  https://github.com/roman-rybalko/telegram-data-clustering-contest/archive/8da391a6b6d952bbf0555e8a80ff864cf9c79bda.zip
Resolving github.com (github.com)... 140.82.118.4
Connecting to github.com (github.com)|140.82.118.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/roman-rybalko/telegram-data-clustering-contest/zip/8da391a6b6d952bbf0555e8a80ff864cf9c79bda [following]
--2019-11-22 20:27:49--  https://codeload.github.com/roman-rybalko/telegram-data-clustering-contest/zip/8da391a6b6d952bbf0555e8a80ff864cf9c79bda
Resolving codeload.github.com (codeload.github.com)... 192.30.253.121
Connecting to codeload.github.com (codeload.github.com)|192.30.253.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘d2.zip’

d2.zip                  [   <=>              ] 982.42M  24.1MB/s    in 97s     

2019-11-22 20:29:27 (10.1 MB/s) - ‘d2.zip’ saved [10

In [2]:
import os
def get_files(d):
  return [os.path.join(dp, f) for dp, dn, fn in os.walk(d) for f in fn]
files = [f for f in get_files('dataset') if os.path.splitext(f)[1] == '.html']
len(files)

467483

In [3]:
!find dataset -name '*.html' | wc -l

467483


In [4]:
!pip install langdetect

In [5]:
# Reinstall pytables for hdf support (google colab has old & glitchy pytables)
#!pip uninstall -y tables
#!pip install tables

Uninstalling tables-3.6.1:
  Successfully uninstalled tables-3.6.1
  Using cached https://files.pythonhosted.org/packages/0f/cb/4097be890a773af95343389faa8c283b0d9ff606f144227a548461dcbdd5/tables-3.6.1-cp37-cp37m-manylinux1_x86_64.whl


In [6]:
import pandas as pd
from bs4 import BeautifulSoup
import langdetect
from tqdm import tqdm
from joblib import Parallel, delayed

def process(fn):
  with open(fn, 'r') as f:
    text = BeautifulSoup(f, 'html.parser').get_text()
  lang = None
  try:
    lang = langdetect.detect(text)
  except:
    pass
  return [fn, text, lang]

df = pd.DataFrame(data = Parallel(n_jobs=-1, verbose=3)(delayed(process)(fn) for fn in tqdm(files)),
                  columns=['filename', 'text', 'lang'])
df.info()

100%|██████████| 467483/467483 [19:12<00:00, 405.51it/s]
[Parallel(n_jobs=-1)]: Done 467483 out of 467483 | elapsed: 19.2min finished


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467483 entries, 0 to 467482
Data columns (total 3 columns):
filename    467483 non-null object
text        467483 non-null object
lang        467385 non-null object
dtypes: object(3)
memory usage: 10.7+ MB


In [7]:
df['lang'].value_counts()

ru       120468
en       107186
es        38144
ar        28138
uk        16790
fr        15367
de        14627
fa        14392
pt        12168
id        12062
it        11338
el         9666
bg         8787
ko         7705
tr         5456
vi         4360
no         3326
nl         3058
ro         2870
ml         2829
ja         2656
hi         2584
cs         2493
hu         2309
th         2144
sl         1902
hr         1755
te         1564
ca         1503
zh-tw      1458
mr         1389
sv         1189
sk         1140
he         1124
ta          750
pl          701
bn          670
zh-cn       504
lt          345
tl          109
sw           82
so           63
mk           54
et           52
ur           50
lv           35
af           15
da            7
sq            1
Name: lang, dtype: int64

In [8]:
df = df[(df['lang'] == 'en') | (df['lang'] == 'ru')]
df.shape

(227654, 3)

In [9]:
df.head()

,filename,text,lang
0,dataset/20191108/15/1491978796208217930.html,\n\n\n\n\n\n\n\n\n\n\n\nNewcastle United v Bou...,en
4,dataset/20191108/15/6359253557838878602.html,\n\n\n\n\n\n\n\n\n\n\n\nМинкомсвязь негативно ...,ru
6,dataset/20191108/15/8549607152500932485.html,\n\n\n\n\n\n\n\n\n\n\n\nTether Responds to Mar...,en
7,dataset/20191108/15/3752801377276103486.html,\n\n\n\n\n\n\n\n\n\n\n\nAnger over Las Vegas b...,en
10,dataset/20191108/15/5545714965965021212.html,\n\n\n\n\n\n\n\n\n\n\n\nAmerican Truck Simulat...,en


In [10]:
%%time
df.to_hdf('fe1.h5', 'fe1')
!ls -l fe1.h5

-rw-r--r-- 1 root root 755315112 Nov 22 20:50 fe1.h5
CPU times: user 1.64 s, sys: 3.03 s, total: 4.66 s
Wall time: 4.81 s


In [11]:
#%%time
#import pandas as pd
#df = pd.read_hdf('fe1.h5')
#df.info()